In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
from scipy import stats

# 2. Load combined CSV file
print("Upload the combined CSV file with unemployment and crime data")
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(BytesIO(uploaded[file_name]))
print(f"File '{file_name}' successfully loaded and read.")

# 3. Transform data to "long" format
year_columns = [str(year) for year in range(1998, 2023)]  # Year columns

# Split data into Unemployment and Crime
df_unemployment = df[df['Index'] == 'Unemployment'].copy()
df_crime = df[df['Index'] == 'Crime'].copy()

print(f"Found {len(df_unemployment)} unemployment records")
print(f"Found {len(df_crime)} crime records")

# Transform Unemployment to "long" format
df_unemployment_melted = pd.melt(
    df_unemployment,
    id_vars=['Country Code', 'Country name'],
    value_vars=year_columns,
    var_name='Year',
    value_name='UnemploymentIndex'
)

# Transform Crime to "long" format
df_crime_melted = pd.melt(
    df_crime,
    id_vars=['Country Code', 'Country name'],
    value_vars=year_columns,
    var_name='Year',
    value_name='CrimeIndex'
)

# Merge data - only records with both indicators
df_combined = pd.merge(
    df_unemployment_melted,
    df_crime_melted,
    on=['Country Code', 'Country name', 'Year'],
    how='inner'  # Keep only records with both indicators
)

# Convert 'Year' to numeric format
df_combined['Year'] = df_combined['Year'].astype(int)

# 4. Map country codes to regions
region_map = {
    'Africa': [
        'AGO', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD', 'COG', 'COM',
        'CPV', 'DJI', 'DZA', 'EGY', 'ERI', 'ETH', 'GAB', 'GHA', 'GIN', 'GMB', 'GNB',
        'GNQ', 'KEN', 'LBR', 'LBY', 'LSO', 'MAR', 'MDG', 'MLI', 'MOZ', 'MRT', 'MUS',
        'MWI', 'NAM', 'NER', 'NGA', 'RWA', 'SDN', 'SEN', 'SLE', 'SOM', 'SSD', 'STP',
        'SWZ', 'SYC', 'TCD', 'TGO', 'TUN', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE'
    ],
    'Europe': [
        'ALB', 'AND', 'AUT', 'BEL', 'BIH', 'BGR', 'BLR', 'CHE', 'CYP', 'CZE', 'DEU',
        'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN', 'IRL', 'ISL',
        'ITA', 'KOS', 'LIE', 'LTU', 'LUX', 'LVA', 'MCO', 'MDA', 'MKD', 'MLT', 'MNE',
        'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SMR', 'SRB', 'SVK', 'SVN', 'SWE',
        'UKR', 'VAT', 'XKX'
    ],
    'Asia': [
        'AFG', 'ARM', 'AZE', 'BHR', 'BGD', 'BRN', 'BTN', 'CHN', 'CYP', 'GEO', 'HKG',
        'IDN', 'IND', 'IRN', 'IRQ', 'ISR', 'JOR', 'JPN', 'KAZ', 'KGZ', 'KHM', 'KOR',
        'KWT', 'LAO', 'LBN', 'LKA', 'MAC', 'MDV', 'MMR', 'MNG', 'MYS', 'NPL', 'OMN',
        'PAK', 'PHL', 'PRK', 'PSE', 'QAT', 'SAU', 'SGP', 'SYR', 'THA', 'TJK', 'TKM',
        'TLS', 'TUR', 'UZB', 'VNM', 'YEM'
    ],

      # I did not take these regions through the small number of countries in them but you can add them

    # 'North America': [
    #     'USA', 'CAN', 'BMU', 'GRL', 'CYM', 'TCA'
    # ],
    # 'South America': [
    #     'ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'GUY', 'PRY', 'PER', 'SUR', 'URY', 'VEN'
    # ],
    # 'Oceania': [
    #     'AUS', 'FJI', 'FSM', 'KIR', 'MHL', 'NRU', 'NZL', 'PLW', 'PNG', 'SLB', 'TON',
    #     'TUV', 'VUT', 'WSM'
    # ]
}

# Function to determine region
def get_region(country_code):
    for region, countries in region_map.items():
        if country_code in countries:
            return region
    return None

# Add region column
df_combined['Region'] = df_combined['Country Code'].apply(get_region)

# Function to clean numeric values
def clean_numeric(value):
    if pd.isna(value):
        return np.nan

    str_val = str(value).strip()

    # Check for invalid formats (like "10.05.25")
    if '.' in str_val and len(str_val.split('.')) > 2:
        return np.nan

    try:
        return float(str_val)
    except ValueError:
        return np.nan

df_combined['UnemploymentIndex'] = df_combined['UnemploymentIndex'].apply(clean_numeric)
df_combined['CrimeIndex'] = df_combined['CrimeIndex'].apply(clean_numeric)

# Remove rows with NaN values or without assigned region
df_clean = df_combined.dropna(subset=['UnemploymentIndex', 'CrimeIndex', 'Region'])

print(f"After cleaning, {len(df_clean)} records remain")

# 5. CUSTOM MATHEMATICAL FUNCTIONS

import math

# 1. Manual descriptive statistics
def describe_manual(data):
    """Calculate mean, median, variance (population), std, min, max manually."""
    n = len(data)
    if n == 0:
        return None
    sorted_data = sorted(data)  # Sort for median, min, max
    mean_val = sum(data) / n
    # median
    if n % 2 == 1:
        median_val = sorted_data[n//2]
    else:
        median_val = (sorted_data[n//2 - 1] + sorted_data[n//2]) / 2
    # variance and std (for population)
    var_val = sum((x - mean_val)**2 for x in data) / n
    std_val = math.sqrt(var_val)
    min_val = sorted_data[0]
    max_val = sorted_data[-1]
    return mean_val, median_val, var_val, std_val, min_val, max_val

# 2. IQR analysis manually (for outlier detection)
def iqr_outliers(data):
    """Calculate Q1, Q3, IQR and find outliers using 1.5*IQR rule."""
    if not data:
        return [], None, None, None
    sorted_data = sorted(data)
    n = len(sorted_data)
    # Find median and split into two halves
    if n % 2 == 1:
        lower = sorted_data[:n//2]
        upper = sorted_data[n//2+1:]
    else:
        lower = sorted_data[:n//2]
        upper = sorted_data[n//2:]
    # Function to calculate median of a list
    def median(lst):
        m = len(lst)
        return (lst[m//2] if m%2 else (lst[m//2 - 1] + lst[m//2]) / 2) if m>0 else None
    Q1 = median(lower)
    Q3 = median(upper)
    if Q1 is None or Q3 is None:
        return [], Q1, Q3, None
    IQR = Q3 - Q1
    lower_fence = Q1 - 1.5 * IQR
    upper_fence = Q3 + 1.5 * IQR
    outliers = [x for x in data if x < lower_fence or x > upper_fence]
    return outliers, Q1, Q3, IQR

# 3. Spearman correlation with p-value
def spearman_correlation_with_pvalue(x, y):
    """Spearman correlation with p-value"""
    n = len(x)
    if n < 3:
        return None, None

    # Ranking function
    def rankdata(arr):
        # Create list of pairs (value, original index)
        indexed_values = [(val, idx) for idx, val in enumerate(arr)]
        # Sort by value
        indexed_values.sort(key=lambda x: x[0])

        ranks = [0] * len(arr)
        i = 0

        while i < len(indexed_values):
            # Find all elements with the same value
            current_value = indexed_values[i][0]
            j = i
            while j < len(indexed_values) and indexed_values[j][0] == current_value:
                j += 1

            # Calculate average rank for tied values
            avg_rank = (2 * i + j - i + 1) / 2.0  # Average of ranks

            # Assign average rank to all tied values
            for k in range(i, j):
                original_index = indexed_values[k][1]
                ranks[original_index] = avg_rank

            i = j

        return ranks

    # Calculate ranks
    rank_x = rankdata(x)
    rank_y = rankdata(y)

    # Calculate Pearson correlation for ranks
    def pearson_for_ranks(rx, ry):
        n = len(rx)
        mean_rx = sum(rx) / n
        mean_ry = sum(ry) / n

        numerator = sum((rx[i] - mean_rx) * (ry[i] - mean_ry) for i in range(n))
        sum_sq_x = sum((rx[i] - mean_rx)**2 for i in range(n))
        sum_sq_y = sum((ry[i] - mean_ry)**2 for i in range(n))

        if sum_sq_x == 0 or sum_sq_y == 0:
            return 0.0

        return numerator / math.sqrt(sum_sq_x * sum_sq_y)

    rho = pearson_for_ranks(rank_x, rank_y)

    # Calculate p-value for Spearman correlation
    if abs(rho) >= 0.999:
        return rho, 0.0

    df = n - 2
    if df <= 0:
        return rho, 1.0

    # t-statistic for Spearman correlation
    try:
        t_stat = rho * math.sqrt(df / (1 - rho**2))
    except (ZeroDivisionError, ValueError):
        return rho, 0.0 if abs(rho) > 0.8 else 1.0

    # Calculate p-value
    def spearman_pvalue(t, df):
        """P-value for Spearman correlation"""
        abs_t = abs(t)

        if df >= 30:
            # For large samples, use normal approximation
            if abs_t > 3.3:
                return 0.001
            elif abs_t > 2.6:
                return 0.01
            elif abs_t > 2.0:
                return 0.05
            elif abs_t > 1.6:
                return 0.1
            elif abs_t > 1.0:
                return 0.3
            else:
                return 0.6
        else:
            # For small samples, more conservative estimates
            if abs_t > 3.0:
                return 0.01
            elif abs_t > 2.5:
                return 0.05
            elif abs_t > 2.0:
                return 0.1
            elif abs_t > 1.5:
                return 0.2
            elif abs_t > 1.0:
                return 0.4
            else:
                return 0.7

    p_val = spearman_pvalue(t_stat, df)
    return rho, min(p_val, 1.0)

# 4. Linear regression manually
def linear_regression(x, y):
    """Calculate α (intercept), β (slope) and R² without sklearn/statsmodels."""
    n = len(x)
    mean_x = sum(x)/n
    mean_y = sum(y)/n
    cov = sum((xi - mean_x)*(yi - mean_y) for xi, yi in zip(x, y))
    var_x = sum((xi - mean_x)**2 for xi in x)
    if var_x == 0:
        return None
    beta = cov / var_x
    alpha = mean_y - beta * mean_x

    ss_tot = sum((yi - mean_y)**2 for yi in y)
    ss_reg = sum((beta*xi + alpha - mean_y)**2 for xi in x)
    r2 = ss_reg / ss_tot if ss_tot != 0 else None
    return alpha, beta, r2

# Helper functions
def calculate_mean(values):
    """Arithmetic mean"""
    result = describe_manual(values)
    return result[0] if result else 0

def calculate_median(values):
    """Median"""
    result = describe_manual(values)
    return result[1] if result else 0

def calculate_variance(values, sample=True):
    """Variance"""
    result = describe_manual(values)
    if result is None:
        return 0
    if sample and len(values) > 1:
        # Convert from population to sample variance
        return result[2] * len(values) / (len(values) - 1)
    return result[2]

def calculate_std(values, sample=True):
    """Standard deviation"""
    return math.sqrt(calculate_variance(values, sample))

def detect_outliers_iqr(values):
    """Detect anomalies using IQR method"""
    if len(values) <= 2:
        return [], values
    outliers, q1, q3, iqr = iqr_outliers(values)
    normal_values = [x for x in values if x not in outliers]
    return outliers, normal_values

def replace_outliers_with_median(values):
    """Replace anomalies with median"""
    if len(values) <= 2:
        return values

    outliers, normal_values = detect_outliers_iqr(values)

    if len(normal_values) == 0:
        return values

    median_val = calculate_median(normal_values)

    result = []
    for val in values:
        if val in outliers:
            result.append(median_val)
        else:
            result.append(val)

    return result

def calculate_skewness(values):
    """Coefficient of asymmetry (skewness)"""
    if len(values) < 3:
        return 0

    n = len(values)
    mean_val = calculate_mean(values)
    std_val = calculate_std(values, sample=False)

    if std_val == 0:
        return 0

    skew_sum = sum(((x - mean_val) / std_val) ** 3 for x in values)
    return skew_sum / n

def calculate_kurtosis(values):
    """Coefficient of excess kurtosis"""
    if len(values) < 4:
        return 0

    n = len(values)
    mean_val = calculate_mean(values)
    std_val = calculate_std(values, sample=False)

    if std_val == 0:
        return 0

    kurt_sum = sum(((x - mean_val) / std_val) ** 4 for x in values)
    return (kurt_sum / n) - 3  # Excess kurtosis relative to normal distribution

def test_normality(values):
    """Simple test for distribution normality"""
    if len(values) < 8:
        return "Insufficient data", True  # Assume normality

    skew = calculate_skewness(values)
    kurt = calculate_kurtosis(values)

    # Criteria for normality (approximate)
    is_normal = abs(skew) < 2 and abs(kurt) < 2

    interpretation = []
    if abs(skew) >= 2:
        interpretation.append(f"High asymmetry ({skew:.2f})")
    if abs(kurt) >= 2:
        interpretation.append(f"High kurtosis ({kurt:.2f})")

    if is_normal:
        return "Approximately normal", True
    else:
        return f"Not normal: {', '.join(interpretation)}", False

def calculate_spearman_correlation(x_values, y_values):
    """Spearman correlation coefficient with p-value"""
    if len(x_values) != len(y_values) or len(x_values) < 2:
        return 0, None

    result = spearman_correlation_with_pvalue(x_values, y_values)
    if result[0] is None:
        return 0, None
    return result[0], result[1]

# 6. ANALYSIS BY REGIONS AND YEARS (WITH USER PROMPT)
print("\n=== ANALYSIS BY REGIONS AND YEARS ===")

# Ask user if they want to see detailed statistics
show_details = input("\nDo you want to display detailed yearly statistics for each region? (yes/no): ").strip().lower()

regions = df_clean['Region'].unique()
years = sorted(df_clean['Year'].unique())

# Dictionaries to store results
regional_stats = {}
correlation_results = {}
regression_results = {}
normality_results = {}

for region in regions:
    if show_details == 'yes':
        print(f"\n=== REGION: {region} ===")

    regional_stats[region] = {}
    correlation_results[region] = {}
    regression_results[region] = {}
    normality_results[region] = {}

    region_data = df_clean[df_clean['Region'] == region]

    for year in years:
        year_data = region_data[region_data['Year'] == year]

        # Check if there's data for this year
        if len(year_data) == 0:
            continue

        if show_details == 'yes':
            print(f"\n--- Year {year} ---")
            print(f"Number of countries with both coefficients: {len(year_data)}")

        # Get values
        unemp_vals = year_data['UnemploymentIndex'].values.tolist()
        crime_vals = year_data['CrimeIndex'].values.tolist()

        # Test distributions
        unemp_normality, unemp_is_normal = test_normality(unemp_vals)
        crime_normality, crime_is_normal = test_normality(crime_vals)

        if show_details == 'yes':
            print(f"Unemployment distribution: {unemp_normality}")
            print(f"Crime distribution: {crime_normality}")

        normality_results[region][year] = {
            'unemployment_normal': unemp_is_normal,
            'crime_normal': crime_is_normal
        }

        # Detect and handle anomalies
        if show_details == 'yes':
            print("Anomaly analysis:")
        unemp_outliers, unemp_normal = detect_outliers_iqr(unemp_vals)
        crime_outliers, crime_normal = detect_outliers_iqr(crime_vals)

        if show_details == 'yes':
            print(f"  Unemployment: {len(unemp_outliers)} anomalies out of {len(unemp_vals)} values")
            print(f"  Crime: {len(crime_outliers)} anomalies out of {len(crime_vals)} values")

        # Replace anomalies with median
        unemp_cleaned = replace_outliers_with_median(unemp_vals)
        crime_cleaned = replace_outliers_with_median(crime_vals)

        # Descriptive statistics
        unemp_stats = describe_manual(unemp_cleaned)
        crime_stats = describe_manual(crime_cleaned)

        stats = {
            'unemployment': {
                'mean': unemp_stats[0] if unemp_stats else 0,
                'median': unemp_stats[1] if unemp_stats else 0,
                'variance': calculate_variance(unemp_cleaned),
                'std': calculate_std(unemp_cleaned),
                'min': unemp_stats[4] if unemp_stats else 0,
                'max': unemp_stats[5] if unemp_stats else 0,
                'skewness': calculate_skewness(unemp_cleaned),
                'kurtosis': calculate_kurtosis(unemp_cleaned)
            },
            'crime': {
                'mean': crime_stats[0] if crime_stats else 0,
                'median': crime_stats[1] if crime_stats else 0,
                'variance': calculate_variance(crime_cleaned),
                'std': calculate_std(crime_cleaned),
                'min': crime_stats[4] if crime_stats else 0,
                'max': crime_stats[5] if crime_stats else 0,
                'skewness': calculate_skewness(crime_cleaned),
                'kurtosis': calculate_kurtosis(crime_cleaned)
            }
        }

        regional_stats[region][year] = stats

        if show_details == 'yes':
            print("Descriptive statistics (after handling anomalies):")
            print(f"  Unemployment: μ={stats['unemployment']['mean']:.2f}, Med={stats['unemployment']['median']:.2f}, σ={stats['unemployment']['std']:.2f}")
            print(f"  Crime: μ={stats['crime']['mean']:.2f}, Med={stats['crime']['median']:.2f}, σ={stats['crime']['std']:.2f}")

        # Calculate Spearman correlation
        correlation, p_value_spearman = calculate_spearman_correlation(unemp_cleaned, crime_cleaned)

        correlation_results[region][year] = {
            'correlation': correlation,
            'p_value': p_value_spearman
        }

        if show_details == 'yes':
            print(f"Spearman Correlation: r = {correlation:.3f}")
            if p_value_spearman is not None:
                significance = "***" if p_value_spearman < 0.001 else "**" if p_value_spearman < 0.01 else "*" if p_value_spearman < 0.05 else ""
                print(f"p-value: {p_value_spearman:.6f} {significance}")

        # Linear regression
        if len(unemp_cleaned) >= 2:
            reg_result = linear_regression(unemp_cleaned, crime_cleaned)
            if reg_result is not None:
                a, b, r_squared = reg_result

                regression_results[region][year] = {
                    'intercept': a,
                    'slope': b,
                    'r_squared': r_squared if r_squared is not None else 0
                }

                if show_details == 'yes':
                    print(f"Linear regression: Crime = {a:.3f} + {b:.3f} * Unemployment")
                    print(f"Coefficient of determination: R² = {r_squared:.3f}" if r_squared is not None else "R² = N/A")

if show_details != 'yes':
    print("\nDetailed statistics skipped. Proceeding to visualizations...\n")

# 7. INDEX DISTRIBUTION HISTOGRAMS FOR EACH REGION
print("\n=== INDEX DISTRIBUTION HISTOGRAMS ===")

plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 10)

for region in regions:
    region_data = df_clean[df_clean['Region'] == region]

    if len(region_data) == 0:
        continue

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Index Distributions: {region}', fontsize=16, fontweight='bold')

    # Unemployment histogram
    unemp_all = region_data['UnemploymentIndex'].values
    ax1.hist(unemp_all, bins=20, alpha=0.7, color='blue', edgecolor='black')
    ax1.set_title('Unemployment Index Distribution')
    ax1.set_xlabel('Unemployment Index')
    ax1.set_ylabel('Frequency')
    ax1.grid(True, alpha=0.3)

    # Statistics for unemployment
    unemp_stats = describe_manual(unemp_all.tolist())
    if unemp_stats:
        unemp_mean, unemp_median = unemp_stats[0], unemp_stats[1]
        ax1.axvline(unemp_mean, color='red', linestyle='--', label=f'Mean: {unemp_mean:.2f}')
        ax1.axvline(unemp_median, color='green', linestyle='--', label=f'Median: {unemp_median:.2f}')
        ax1.legend()

    # Crime histogram
    crime_all = region_data['CrimeIndex'].values
    ax2.hist(crime_all, bins=20, alpha=0.7, color='red', edgecolor='black')
    ax2.set_title('Crime Index Distribution')
    ax2.set_xlabel('Crime Index')
    ax2.set_ylabel('Frequency')
    ax2.grid(True, alpha=0.3)

    # Statistics for crime
    crime_stats = describe_manual(crime_all.tolist())
    if crime_stats:
        crime_mean, crime_median = crime_stats[0], crime_stats[1]
        ax2.axvline(crime_mean, color='red', linestyle='--', label=f'Mean: {crime_mean:.2f}')
        ax2.axvline(crime_median, color='green', linestyle='--', label=f'Median: {crime_median:.2f}')
        ax2.legend()

    # Q-Q plot for unemployment
    unemp_sorted = sorted(unemp_all)
    n = len(unemp_sorted)
    theoretical_quantiles = [(i - 0.5) / n for i in range(1, n + 1)]
    norm_quantiles = [math.sqrt(2) * (2 * q - 1) for q in theoretical_quantiles]

    ax3.scatter(norm_quantiles, unemp_sorted, alpha=0.6)
    ax3.plot([min(norm_quantiles), max(norm_quantiles)],
             [min(unemp_sorted), max(unemp_sorted)], 'r--')
    ax3.set_title('Q-Q plot: Unemployment vs Normal Distribution')
    ax3.set_xlabel('Theoretical Quantiles')
    ax3.set_ylabel('Sample Quantiles')
    ax3.grid(True, alpha=0.3)

    # Q-Q plot for crime
    crime_sorted = sorted(crime_all)
    ax4.scatter(norm_quantiles[:len(crime_sorted)], crime_sorted, alpha=0.6, color='red')
    ax4.plot([min(norm_quantiles[:len(crime_sorted)]), max(norm_quantiles[:len(crime_sorted)])],
             [min(crime_sorted), max(crime_sorted)], 'r--')
    ax4.set_title('Q-Q plot: Crime vs Normal Distribution')
    ax4.set_xlabel('Theoretical Quantiles')
    ax4.set_ylabel('Sample Quantiles')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Output distribution statistics
    print(f"\n{region} - Distribution Statistics:")
    unemp_skew = calculate_skewness(unemp_all.tolist())
    unemp_kurt = calculate_kurtosis(unemp_all.tolist())
    crime_skew = calculate_skewness(crime_all.tolist())
    crime_kurt = calculate_kurtosis(crime_all.tolist())

    print(f"Unemployment: skewness={unemp_skew:.3f}, kurtosis={unemp_kurt:.3f}")
    print(f"Crime: skewness={crime_skew:.3f}, kurtosis={crime_kurt:.3f}")

# 8. CORRELATION HEATMAP
print("\n=== CORRELATION HEATMAP ===")

# Create correlation matrix by regions and years
correlation_matrix_data = []
regions_list = sorted(regions)
years_list = sorted(years)

for region in regions_list:
    row = []
    for year in years_list:
        if region in correlation_results and year in correlation_results[region]:
            corr_val = correlation_results[region][year]['correlation']
            row.append(corr_val)
        else:
            row.append(np.nan)
    correlation_matrix_data.append(row)

correlation_matrix = np.array(correlation_matrix_data)

# Create heatmap
plt.figure(figsize=(20, 8))
mask = np.isnan(correlation_matrix)
sns.heatmap(correlation_matrix,
            xticklabels=years_list,
            yticklabels=regions_list,
            annot=True,
            fmt='.3f',
            cmap='RdBu_r',
            center=0,
            mask=mask,
            cbar_kws={'label': 'Spearman Correlation Coefficient'},
            square=False)

plt.title('Heatmap of Spearman Correlations between Unemployment and Crime\n(by Regions and Years)',
          fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Region', fontsize=12)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Additional heatmap - average correlations by regions
print("\n=== AVERAGE CORRELATIONS BY REGIONS HEATMAP ===")

avg_correlations = []
region_names = []

for region in regions_list:
    if region in correlation_results and len(correlation_results[region]) > 0:
        correlations = [data['correlation'] for data in correlation_results[region].values()
                       if data['correlation'] is not None and not np.isnan(data['correlation'])]
        if correlations:
            avg_corr = calculate_mean(correlations)
            avg_correlations.append([avg_corr])
            region_names.append(f"{region}\n(n={len(correlations)})")

if avg_correlations:
    plt.figure(figsize=(3, 8))
    sns.heatmap(avg_correlations,
                yticklabels=region_names,
                annot=True,
                fmt='.3f',
                cmap='RdBu_r',
                center=0,
                cbar_kws={'label': 'Average Spearman Correlation'},
                square=False)

    plt.title('Average Spearman Correlations by Region', fontsize=14, fontweight='bold')
    plt.xlabel('')
    plt.ylabel('Region (number of years)', fontsize=12)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# 9. RESULTS VISUALIZATION
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

# 9.1 Correlation graphs by years for each region
for region in regions:
    if region not in correlation_results or len(correlation_results[region]) == 0:
        continue

    years_with_data = sorted(correlation_results[region].keys())
    correlations = [correlation_results[region][year]['correlation'] for year in years_with_data]

    plt.figure(figsize=(12, 6))
    plt.plot(years_with_data, correlations, marker='o', linewidth=2, markersize=8)
    plt.title(f'Spearman Correlation by Years: {region}')
    plt.xlabel('Year')
    plt.ylabel('Spearman Correlation Coefficient')
    plt.grid(True, alpha=0.3)
    plt.ylim(-1, 1)
    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

    # Add horizontal lines for interpretation
    plt.axhline(y=0.3, color='green', linestyle=':', alpha=0.7, label='Weak Positive')
    plt.axhline(y=-0.3, color='red', linestyle=':', alpha=0.7, label='Weak Negative')
    plt.axhline(y=0.7, color='darkgreen', linestyle=':', alpha=0.7, label='Strong Positive')
    plt.axhline(y=-0.7, color='darkred', linestyle=':', alpha=0.7, label='Strong Negative')

    # Add number of countries for each point
    for i, (year, corr) in enumerate(zip(years_with_data, correlations)):
        region_year_data = df_clean[(df_clean['Region'] == region) & (df_clean['Year'] == year)]
        n_countries = len(region_year_data)
        plt.annotate(f'n={n_countries}',
                    (year, corr),
                    textcoords="offset points",
                    xytext=(0,10),
                    ha='center',
                    fontsize=8,
                    alpha=0.7)

    plt.legend()
    plt.tight_layout()
    plt.show()

# 9.2 Scatter plots with regression for the latest year with data
print("\n=== SCATTER PLOTS WITH REGRESSION (Latest Year with Data) ===")

for region in regions:
    region_data = df_clean[df_clean['Region'] == region]

    if len(region_data) == 0:
        continue

    # Find latest year with data
    latest_year = None
    for year in sorted(region_data['Year'].unique(), reverse=True):
        year_data = region_data[region_data['Year'] == year]
        if len(year_data) >= 1:
            latest_year = year
            break

    if latest_year is None:
        continue

    year_data = region_data[region_data['Year'] == latest_year]

    # Handle anomalies
    unemp_vals = year_data['UnemploymentIndex'].values.tolist()
    crime_vals = year_data['CrimeIndex'].values.tolist()
    unemp_cleaned = replace_outliers_with_median(unemp_vals)
    crime_cleaned = replace_outliers_with_median(crime_vals)

    plt.figure(figsize=(10, 8))

    # Different markers depending on number of countries
    if len(year_data) == 1:
        plt.scatter(unemp_cleaned, crime_cleaned, alpha=0.8, s=200, c='red',
                   marker='*', label=f'Data from {latest_year} (1 country)')
    else:
        plt.scatter(unemp_cleaned, crime_cleaned, alpha=0.7, s=80,
                   label=f'Data from {latest_year} ({len(year_data)} countries)')

    # Add regression line only if more than 1 point
    if len(year_data) > 1 and latest_year in regression_results.get(region, {}):
        reg_params = regression_results[region][latest_year]
        a, b = reg_params['intercept'], reg_params['slope']
        r_squared = reg_params['r_squared']

        x_min, x_max = min(unemp_cleaned), max(unemp_cleaned)
        x_line = [x_min, x_max]
        y_line = [a + b * x for x in x_line]

        plt.plot(x_line, y_line, color='red', linewidth=2,
                label=f'Regression: y = {a:.2f} + {b:.2f}x\nR² = {r_squared:.3f}')

    # Add country labels
    for i, row in year_data.iterrows():
        plt.annotate(row['Country Code'],
                    (unemp_cleaned[year_data.index.get_loc(i)],
                     crime_cleaned[year_data.index.get_loc(i)]),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=8, alpha=0.7)

    plt.title(f'{region}: Unemployment vs Crime ({latest_year})')
    plt.xlabel('Unemployment Index')
    plt.ylabel('Crime Index')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# 10. INDEX GRAPHS BY YEARS FOR EACH REGION
print("\n=== INDEX GRAPHS BY YEARS FOR EACH REGION ===")

for region in regions:
    region_data = df_clean[df_clean['Region'] == region]

    if len(region_data) == 0:
        continue

    # Group data by years
    yearly_stats = {}
    for year in sorted(region_data['Year'].unique()):
        year_data = region_data[region_data['Year'] == year]
        if len(year_data) > 0:
            unemp_vals = year_data['UnemploymentIndex'].values
            crime_vals = year_data['CrimeIndex'].values

            yearly_stats[year] = {
                'unemployment_mean': calculate_mean(unemp_vals.tolist()),
                'unemployment_median': calculate_median(unemp_vals.tolist()),
                'crime_mean': calculate_mean(crime_vals.tolist()),
                'crime_median': calculate_median(crime_vals.tolist()),
                'count': len(year_data)
            }

    if len(yearly_stats) == 0:
        continue

    years_list = sorted(yearly_stats.keys())
    unemp_means = [yearly_stats[year]['unemployment_mean'] for year in years_list]
    unemp_medians = [yearly_stats[year]['unemployment_median'] for year in years_list]
    crime_means = [yearly_stats[year]['crime_mean'] for year in years_list]
    crime_medians = [yearly_stats[year]['crime_median'] for year in years_list]

    plt.figure(figsize=(14, 8))

    # Create two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # Unemployment graph
    ax1.plot(years_list, unemp_means, marker='o', linewidth=2, label='Mean', color='blue')
    ax1.plot(years_list, unemp_medians, marker='s', linewidth=2, label='Median',
             color='lightblue', linestyle='--')
    ax1.set_title(f'{region}: Unemployment Index by Years')
    ax1.set_ylabel('Unemployment Index')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Add number of countries for each point
    for i, year in enumerate(years_list):
        count = yearly_stats[year]['count']
        ax1.annotate(f'n={count}', (year, unemp_means[i]),
                    textcoords="offset points", xytext=(0,10),
                    ha='center', fontsize=8, alpha=0.7)

    # Crime graph
    ax2.plot(years_list, crime_means, marker='o', linewidth=2, label='Mean', color='red')
    ax2.plot(years_list, crime_medians, marker='s', linewidth=2, label='Median',
             color='pink', linestyle='--')
    ax2.set_title(f'{region}: Crime Index by Years')
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Crime Index')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # Add number of countries for each point
    for i, year in enumerate(years_list):
        count = yearly_stats[year]['count']
        ax2.annotate(f'n={count}', (year, crime_means[i]),
                    textcoords="offset points", xytext=(0,10),
                    ha='center', fontsize=8, alpha=0.7)

    plt.tight_layout()
    plt.show()

    print(f"\n{region} - Statistics by Years:")
    print("Year\tUnemp(mean)\tUnemp(med)\tCrime(mean)\tCrime(med)\tCountries")
    for year in years_list:
        stats = yearly_stats[year]
        print(f"{year}\t{stats['unemployment_mean']:.2f}\t\t{stats['unemployment_median']:.2f}\t\t"
              f"{stats['crime_mean']:.2f}\t\t{stats['crime_median']:.2f}\t\t{stats['count']}")

# 11. COMBINED GRAPHS OF UNEMPLOYMENT AND CRIME BY REGIONS
print("\n=== COMBINED GRAPHS OF UNEMPLOYMENT AND CRIME BY REGIONS ===")

# Collect all data by regions for overall analysis
overall_stats = {}

for region in regions:
    region_data = df_clean[df_clean['Region'] == region]

    if len(region_data) == 0:
        continue

    # Group data by years
    yearly_stats = {}
    for year in sorted(region_data['Year'].unique()):
        year_data = region_data[region_data['Year'] == year]
        if len(year_data) > 0:
            unemp_vals = year_data['UnemploymentIndex'].values
            crime_vals = year_data['CrimeIndex'].values

            yearly_stats[year] = {
                'unemployment_mean': calculate_mean(unemp_vals.tolist()),
                'crime_mean': calculate_mean(crime_vals.tolist()),
                'count': len(year_data)
            }

    if len(yearly_stats) == 0:
        continue

    years_list = sorted(yearly_stats.keys())
    unemp_means = [yearly_stats[year]['unemployment_mean'] for year in years_list]
    crime_means = [yearly_stats[year]['crime_mean'] for year in years_list]

    # Create combined graph
    fig, ax1 = plt.subplots(figsize=(14, 8))

    # Determine common limits for both Y axes
    all_values = unemp_means + crime_means
    global_min = min(all_values)
    global_max = max(all_values)
    padding = (global_max - global_min) * 0.1
    y_min = global_min - padding
    y_max = global_max + padding

    # Unemployment graph (left Y axis)
    color1 = 'tab:blue'
    ax1.set_xlabel('Year', fontsize=12)
    ax1.set_ylabel('Unemployment Index', color=color1, fontsize=12)
    line1 = ax1.plot(years_list, unemp_means, marker='o', linewidth=3,
                     label='Unemployment', color=color1, markersize=8)
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(y_min, y_max)

    # Create second Y axis for crime
    ax2 = ax1.twinx()
    color2 = 'tab:red'
    ax2.set_ylabel('Crime Index', color=color2, fontsize=12)
    line2 = ax2.plot(years_list, crime_means, marker='s', linewidth=3,
                     label='Crime', color=color2, markersize=8, linestyle='--')
    ax2.tick_params(axis='y', labelcolor=color2)
    ax2.set_ylim(y_min, y_max)

    # Add title
    plt.title(f'{region}: Dynamics of Unemployment and Crime Indices',
              fontsize=16, fontweight='bold', pad=20)

    # Add legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=11)

    # Add number of countries for each point
    for i, year in enumerate(years_list):
        count = yearly_stats[year]['count']
        ax1.annotate(f'n={count}', (year, unemp_means[i]),
                    textcoords="offset points", xytext=(0,15),
                    ha='center', fontsize=9, alpha=0.8, color=color1,
                    weight='bold')

    plt.tight_layout()
    plt.show()

    # Calculate overall correlation for this region
    all_unemp = region_data['UnemploymentIndex'].values.tolist()
    all_crime = region_data['CrimeIndex'].values.tolist()
    unemp_cleaned = replace_outliers_with_median(all_unemp)
    crime_cleaned = replace_outliers_with_median(all_crime)
    overall_corr, overall_pvalue = calculate_spearman_correlation(unemp_cleaned, crime_cleaned)

    # Output brief report
    print(f"\n{region} - Brief Report:")
    if len(years_list) >= 2:
        unemp_trend = "increase" if unemp_means[-1] > unemp_means[0] else "decrease"
        crime_trend = "increase" if crime_means[-1] > crime_means[0] else "decrease"
        print(f"  Unemployment trend: {unemp_trend} ({unemp_means[0]:.2f} -> {unemp_means[-1]:.2f})")
        print(f"  Crime trend: {crime_trend} ({crime_means[0]:.2f} -> {crime_means[-1]:.2f})")
        print(f"  Overall correlation: {overall_corr:.3f} ({'positive' if overall_corr > 0 else 'negative'})")

    print(f"  Period: {min(years_list)}-{max(years_list)} ({len(years_list)} years)")
    print(f"  Average countries per year: {calculate_mean([yearly_stats[y]['count'] for y in years_list]):.1f}")

# 12. SUMMARY TABLES
print("\n=== SUMMARY RESULTS WITH P-VALUE ===")

# Collect all data by regions for overall analysis
overall_stats = {}

for region in regions:
    if region not in correlation_results or len(correlation_results[region]) == 0:
        continue

    # Collect all data for this region across all years
    region_data = df_clean[df_clean['Region'] == region]
    all_unemp = region_data['UnemploymentIndex'].values.tolist()
    all_crime = region_data['CrimeIndex'].values.tolist()

    # Handle anomalies
    unemp_cleaned = replace_outliers_with_median(all_unemp)
    crime_cleaned = replace_outliers_with_median(all_crime)

    # Calculate overall correlation and p-value
    overall_corr, overall_pvalue = calculate_spearman_correlation(unemp_cleaned, crime_cleaned)

    # Collect yearly correlations
    yearly_correlations = []
    yearly_pvalues = []
    for year_data in correlation_results[region].values():
        yearly_correlations.append(year_data['correlation'])
        if year_data['p_value'] is not None:
            yearly_pvalues.append(year_data['p_value'])

    avg_yearly_correlation = calculate_mean(yearly_correlations)
    avg_yearly_pvalue = calculate_mean(yearly_pvalues) if yearly_pvalues else None

    overall_stats[region] = {
        'overall_correlation': overall_corr,
        'overall_pvalue': overall_pvalue,
        'avg_yearly_correlation': avg_yearly_correlation,
        'avg_yearly_pvalue': avg_yearly_pvalue,
        'years_count': len(yearly_correlations),
        'countries_count': len(region_data['Country Code'].unique()),
        'total_observations': len(region_data)
    }

# Detailed table with p-values
print("\nDetailed Table by Regions with p-value:")
print("Region\t\t\tOverall.corr\tp-value\t\tAvg.yearly\tp-val(yearly)\tYears\tCountries\tObs.")
print("-" * 120)

for region in regions:
    if region not in overall_stats:
        continue

    stats = overall_stats[region]

    # Format p-values with stars for significance
    def format_pvalue(pval):
        if pval is None:
            return "N/A\t"
        significance = "***" if pval < 0.001 else "**" if pval < 0.01 else "* " if pval < 0.05 else ""
        return f"{pval:.4f}{significance}\t"

    overall_pval_str = format_pvalue(stats['overall_pvalue'])
    yearly_pval_str = format_pvalue(stats['avg_yearly_pvalue'])

    print(f"{region:<20}\t{stats['overall_correlation']:.3f}\t\t{overall_pval_str}"
          f"{stats['avg_yearly_correlation']:.3f}\t\t{yearly_pval_str}"
          f"{stats['years_count']}\t{stats['countries_count']}\t{stats['total_observations']}")

# Overall significance assessment
print(f"\n=== OVERALL SIGNIFICANCE ASSESSMENT ===")
significant_regions = []
for region, stats in overall_stats.items():
    if stats['overall_pvalue'] is not None and stats['overall_pvalue'] < 0.05:
        significant_regions.append(region)

print(f"Regions with statistically significant correlation (p < 0.05): {len(significant_regions)}")
for region in significant_regions:
    stats = overall_stats[region]
    strength = "strong" if abs(stats['overall_correlation']) > 0.7 else \
               "moderate" if abs(stats['overall_correlation']) > 0.3 else "weak"
    direction = "positive" if stats['overall_correlation'] > 0 else "negative"
    print(f"   {region}: {strength} {direction} correlation "
          f"(r={stats['overall_correlation']:.3f}, p={stats['overall_pvalue']:.4f})")

# Legend for p-values
print(f"\nSignificance Legend:")
print(f"   *** p < 0.001 (highly significant)")
print(f"   **  p < 0.01  (very significant)")
print(f"   *   p < 0.05  (significant)")
print(f"   no stars: p ≥ 0.05 (not significant)")

# Overall statistics
total_observations = len(df_clean)
total_countries = len(df_clean['Country Code'].unique())
year_range = f"{min(df_clean['Year'])}-{max(df_clean['Year'])}"

print(f"\n=== OVERALL INFORMATION ===")
print(f"Total number of observations: {total_observations}")
print(f"Total number of countries: {total_countries}")
print(f"Analysis period: {year_range}")
print(f"Number of regions: {len(regions)}")
print(f"Regions with significant correlation: {len(significant_regions)}")